In [1]:
import argparse
from utils.ops import count_parameters, save_yaml, load_yaml
import torch
from torch.multiprocessing import Process, freeze_support
from torch.utils.data import DataLoader
import time
from  pathlib import Path
from utils.datasets import TrainDataset, ValDataset#, to_gpu
from pydoc import locate
import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger
from lightning.pytorch.profilers import SimpleProfiler
from models.callbacks import TrainSamples
from tqdm.notebook import trange

In [2]:
exp_n = 1
site = 2

In [3]:
cfg = load_yaml('cfg.yaml')

site_cfg = load_yaml(f'site_{site}.yaml')

training_params = cfg['training_params']
preparation_params = cfg['preparation_params']
experiment_params = cfg['experiments'][f'exp_{exp_n}']
paths_params = cfg['paths']
general_params = cfg['general_params']

experiments_folders = general_params['experiments_folders']

original_opt_imgs = site_cfg['original_data']['opt_imgs']
original_sar_imgs = site_cfg['original_data']['sar_imgs']

experiments_paths = paths_params['experiments']

#create experiment folder structure
exp_path = Path(experiments_paths) / f'exp_{exp_n}'

models_path = exp_path / experiments_folders['models']
logs_path = exp_path / experiments_folders['logs']
visual_path = exp_path / experiments_folders['visual']
predicted_path = exp_path / experiments_folders['predicted']
results_path = exp_path / experiments_folders['results']
visual_logs_path = exp_path / experiments_folders['visual_logs']

prepared_folder = Path(paths_params['prepared_data'])

train_folder = prepared_folder / preparation_params['train_folder']
val_folder = prepared_folder / preparation_params['validation_folder']
prepared_patches_file = prepared_folder / preparation_params['prepared_data']
prepared_patches = load_yaml(prepared_patches_file)


patch_size = general_params['patch_size']
n_classes = general_params['n_classes']

min_val_loss = training_params['min_val_loss']

In [4]:
train_ds = TrainDataset(experiment_params, train_folder, prepared_patches['train'])
val_ds = ValDataset(experiment_params, val_folder, prepared_patches['val'])

In [5]:
count_cloud = 0
for i in trange(len(train_ds)):
    train_sample = train_ds[i]
    if  train_sample[1][1].max().item() > 0.5 :
        count_cloud += 1

p_cloud = 100 * (count_cloud / len(train_ds))
print(f'Percentage of patches with clouds: {p_cloud:.2f}' )

  0%|          | 0/65070 [00:00<?, ?it/s]

Percentage of patches with clouds: 40.58


In [7]:
count_cloud = 0
for i in trange(len(val_ds)):
    train_sample = val_ds[i]
    if  train_sample[1][1].max().item() > 0.5 :
        count_cloud += 1

p_cloud = 100 * (count_cloud / len(val_ds))
print(f'Percentage of patches with clouds: {p_cloud:.2f}' )

  0%|          | 0/42543 [00:00<?, ?it/s]

Percentage of patches with clouds: 34.21
